<a href="https://colab.research.google.com/github/tjmj-uva/DLIP_project/blob/main/Deep_Learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
nih_chest_xrays_data_path = kagglehub.dataset_download('khanfashee/nih-chest-x-ray-14-224x224-resized')

print('Data source import complete.')


100%|██████████| 2.30G/2.30G [01:47<00:00, 22.9MB/s]

Extracting files...


Data source import complete.


# Introduction

look at post kaggle competition predicting

Proven in medical imaging: Many studies show EfficientNet outperforms older CNNs like DenseNet-121 on X-ray, CT, and MRI datasets.

In this project we will build upon on the DannyNet framework to imporve the classification of diffrenet strains of pneumonia using chest X-rays. DannyNet has demonstrated strong performance using DensNet - 121 we aim to enhance it's accuracy as it is based on some older techniques.

**Data**
We will use the NIH ChestX-ray large dataset of 112,120 x-ray images taken from 30,805 patients. It has 14 labels and some of the x-rays are labeled with more than one diagnosis.
https://www.kaggle.com/datasets/nih-chest-xrays/data/data?select=README_CHESTXRAY.pdf

we wanted to better Dannynet and instead of using densenet we would use **efficient netB0/B4**, this is more efficient as it uses less weights and it shows that there is a higher accuracy when used on medical images(source)

We can add **CBAM** which add two types of attention, **channel attention** where it see which featires matter and **spatial attention** which allows the network to focus on a specific part of the image as it might be more informative.

**To imporve robustness**

We would better **data augmentation** (adds to DennyNet)

* geometric augmentations
* RandomResizedCrop(224)
* RandomHorizontalFlip
* RandomRotation(5°)

**Intensity augmentations**

* ColorJitter(brightness=0.1, contrast=0.1)

* RandomGamma → changes exposure

* RandomContrast → simulates different machines

**mixing augmentations (Dennynet doesnt have)**

* Mixup → blends two images + labels

**Better per class threshold tuning???**
what is the best threshold to differentiate classes -> uses different threshold for each class

**better preprocessing**
* Resize consistently (224×224 or 256×256).
* Normalize with ImageNet stats.

We use **Focal loss and AdamW** similarly as they do in densetent -> address class imbalance and provide stable, efficient training.

Then we will compare it with the original DensNet-121 and human predictions if they are any or otherwise with a Bayesian Baseline Gaussian Naive becuase it is quick and simple but usually very accurate.

**We will copmare it based on:**

* Accuracy
* AUROC (Area Under the ROC Curve) → measures ranking ability
* F1 score → measures exact yes/no detection

**still need to add training details**
* Batch size
* Number of epochs
* Learning rate

In [9]:
import os

data_dir = nih_chest_xrays_data_path
print(f"Data directory: {data_dir}")
print(os.listdir(data_dir))


Data directory: /root/.cache/kagglehub/datasets/khanfashee/nih-chest-x-ray-14-224x224-resized/versions/3
['train_val_list_NIH.txt', 'Data_Entry_2017.csv', 'test_list_NIH.txt', 'images-224', 'BBox_List_2017_Official_NIH.csv', 'pretrained_model.h5']


In [21]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve
import numpy as np
from torchvision.models import densenet121, DenseNet121_Weights
from torch.utils.tensorboard import SummaryWriter
import time
from tqdm import tqdm

CONFIG = {
    "model": "dannynet",
    "batch_size": 8, #8
    "learning_rate": 0.00005,
    "epochs": 1, #25
    "num_workers": 2, #2
    "device": "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu",
    "data_dir": "/root/.cache/kagglehub/datasets/khanfashee/nih-chest-x-ray-14-224x224-resized/versions/3",
    "patience": 5, #5
    "seed": 42,
    "image_size": 224,
}

# Define image transformations (consistent with CheXNet)
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

 # Load and modify the model
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
model.classifier = nn.Linear(model.classifier.in_features, 14)
model = model.to(CONFIG["device"])

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Define loss function and optimizer
criterion = FocalLoss(alpha=1, gamma=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-5) #Added weight decay. # betas=(0.9, 0.999) - this is default in pytorch
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1, factor=0.1)


data_path = CONFIG["data_dir"]
csv_file = os.path.join(data_path, "Data_Entry_2017.csv")
df = pd.read_csv(csv_file)

# Get list of all image folders from images_001 to images_012

# Step-by-step debugging
images_folder = os.path.join(data_path, "images-224", "images-224")

print(f"Looking for images in: {images_folder}")
print(f"Folder exists: {os.path.exists(images_folder)}")

# Build the mapping
image_to_folder = {}
if os.path.exists(images_folder):
    all_files = os.listdir(images_folder)
    png_files = [f for f in all_files if f.endswith('.png')]

    print(f"Total files in folder: {len(all_files)}")
    print(f"PNG files found: {len(png_files)}")
    print(f"Sample PNG files: {png_files[:5]}")

    for img_file in png_files:
        image_to_folder[img_file] = images_folder
else:
    print("ERROR: Folder does not exist!")

print(f"\nTotal images mapped: {len(image_to_folder)}")

# Check CSV
print(f"\nCSV before filtering: {len(df)} rows")
print(f"Sample Image Index from CSV: {df['Image Index'].head().tolist()}")

# Filter
df = df[df['Image Index'].isin(image_to_folder.keys())]

print(f"\nCSV after filtering: {len(df)} rows")
print(f"Unique patients: {df['Patient ID'].nunique()}")

# Unique patient IDs
unique_patients = df['Patient ID'].unique()

# Split patients — not rows
train_val_patients, test_patients = train_test_split(
unique_patients, test_size=0.02, random_state=CONFIG["seed"]
)

train_patients, val_patients = train_test_split(
train_val_patients, test_size=0.052, random_state=CONFIG["seed"]
)

#Use those patients to filter full image rows
train_df = df[df['Patient ID'].isin(train_patients)]
val_df   = df[df['Patient ID'].isin(val_patients)]
test_df  = df[df['Patient ID'].isin(test_patients)]


# List of diseases we’re classifying
disease_list = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
    'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
    'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax'
]

# Function to convert label string to a vector
def get_label_vector(labels_str):
    labels = labels_str.split('|')

    if labels == ['No Finding']:
        return [0] * len(disease_list)

    else:
        return [1 if disease in labels else 0 for disease in disease_list]

# Custom Dataset class
class CheXNetDataset(Dataset):
    def __init__(self, dataframe, image_to_folder, transform=None):
        self.dataframe = dataframe
        self.image_to_folder = image_to_folder
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['Image Index']
        folder = self.image_to_folder[img_name]

        img_path = os.path.join(folder, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        labels_str = self.dataframe.iloc[idx]['Finding Labels']
        label_vector = get_label_vector(labels_str)
        labels = torch.tensor(label_vector, dtype=torch.float)

        return image, labels

# Set up DataLoaders with our custom datasets
train_dataset = CheXNetDataset(train_df, image_to_folder, transform=transform_train)
val_dataset = CheXNetDataset(val_df, image_to_folder, transform=transform_test)
test_dataset = CheXNetDataset(test_df, image_to_folder, transform=transform_test)

trainloader = DataLoader(train_dataset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
valloader = DataLoader(val_dataset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
testloader = DataLoader(test_dataset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])


def get_optimal_thresholds(labels, preds):
    thresholds = []
    for i in range(preds.shape[1]):
        precision, recall, thresh = precision_recall_curve(labels[:, i], preds[:, i])
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_threshold = thresh[np.argmax(f1_scores)] if len(thresh) > 0 else 0.5
        thresholds.append(best_threshold)
    return thresholds

def evaluate(model, loader, criterion, device, desc="[Test]"):
    model.eval()
    running_loss = 0.0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc=desc):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            preds = torch.sigmoid(outputs)
            all_labels.append(labels.cpu())
            all_preds.append(preds.cpu())

    all_labels = torch.cat(all_labels).numpy()
    all_preds = torch.cat(all_preds).numpy()
    thresholds = get_optimal_thresholds(all_labels, all_preds)

    preds_binary = np.zeros_like(all_preds)
    for i in range(all_preds.shape[1]):
        preds_binary[:, i] = (all_preds[:, i] > thresholds[i]).astype(int)

    auc_scores = [roc_auc_score(all_labels[:, i], all_preds[:, i]) for i in range(14)]
    f1_scores = [f1_score(all_labels[:, i], preds_binary[:, i]) for i in range(14)]

    avg_auc = np.mean(auc_scores)
    avg_f1 = np.mean(f1_scores)

    for i, disease in enumerate(disease_list):
        print(f"{desc} {disease} AUC: {auc_scores[i]:.4f} | F1: {f1_scores[i]:.4f}")

    print(f"{desc} Avg AUC: {avg_auc:.4f}, Avg F1: {avg_f1:.4f}")

    return {
        "loss": running_loss / len(loader),
        "avg_auc": avg_auc,
        "avg_f1": avg_f1,
        "auc_dict": dict(zip(disease_list, auc_scores)),
        "f1_dict": dict(zip(disease_list, f1_scores)),
        "thresholds": dict(zip(disease_list, thresholds))
    }


# Training function
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} [Train]", leave=True)
    for i, (inputs, labels) in enumerate(progress_bar):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        progress_bar.set_postfix({"loss": running_loss / (i + 1)})
    train_loss = running_loss / len(trainloader)
    return train_loss

def validate(model, valloader, criterion, device):
    return evaluate(model, valloader, criterion, device, desc="[Validate]")

 #  Initialize TensorBoard
timestamp = time.strftime("%Y%m%d-%H%M%S")
run_id = f"chexnet_{timestamp}"
writer = SummaryWriter(log_dir=f'runs/{run_id}')

# Log config as text
config_str = "\n".join([f"{k}: {v}" for k, v in CONFIG.items()])
writer.add_text('Config', config_str)

checkpoint_dir = os.path.join("models", run_id)
os.makedirs(checkpoint_dir, exist_ok=True)

best_val_auc = 0.0
patience_counter = 0


for epoch in range(CONFIG["epochs"]):
    train_loss = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
    val_stats = validate(model, valloader, criterion, CONFIG["device"])
    scheduler.step(val_stats["loss"])

    # Log to TensorBoard
    writer.add_scalar('Loss/train', train_loss, epoch + 1)
    writer.add_scalar('Loss/val', val_stats["loss"], epoch + 1)
    writer.add_scalar('AUC/val', val_stats["avg_auc"], epoch + 1)
    writer.add_scalar('F1/val', val_stats["avg_f1"], epoch + 1)

    # Log per-disease metrics
    for disease, auc in val_stats["auc_dict"].items():
        writer.add_scalar(f'AUC_per_disease/{disease}', auc, epoch + 1)
    for disease, f1 in val_stats["f1_dict"].items():
        writer.add_scalar(f'F1_per_disease/{disease}', f1, epoch + 1)

    if val_stats["avg_auc"] > best_val_auc:
        best_val_auc = val_stats["avg_auc"]

        patience_counter = 0
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        checkpoint_path = os.path.join(checkpoint_dir, f"best_model_{timestamp}.pth")
        torch.save(model.state_dict(), checkpoint_path)
    else:
        patience_counter += 1
        if patience_counter >= CONFIG["patience"]:
            print("Early stopping triggered.")
            break

# Evaluate the best model
best_checkpoint_path = sorted([os.path.join(checkpoint_dir, f) for f in os.listdir(checkpoint_dir) if f.startswith('best_model_')])[-1]
model.load_state_dict(torch.load(best_checkpoint_path))
test_stats = evaluate(model, testloader, criterion, CONFIG["device"])

# Log test results to TensorBoard
writer.add_scalar('Test/loss', test_stats["loss"], 0)
writer.add_scalar('Test/auc', test_stats["avg_auc"], 0)
writer.add_scalar('Test/f1', test_stats["avg_f1"], 0)

for disease, auc in test_stats["auc_dict"].items():
    writer.add_scalar(f'Test_AUC/{disease}', auc, 0)
for disease, f1 in test_stats["f1_dict"].items():
    writer.add_scalar(f'Test_F1/{disease}', f1, 0)

writer.close()

Looking for images in: /root/.cache/kagglehub/datasets/khanfashee/nih-chest-x-ray-14-224x224-resized/versions/3/images-224/images-224
Folder exists: True
Total files in folder: 112120
PNG files found: 112120
Sample PNG files: ['00023179_006.png', '00017698_008.png', '00016220_021.png', '00013141_000.png', '00010825_001.png']

Total images mapped: 112120

CSV before filtering: 112120 rows
Sample Image Index from CSV: ['00000001_000.png', '00000001_001.png', '00000001_002.png', '00000002_000.png', '00000003_000.png']

CSV after filtering: 112120 rows
Unique patients: 30805


[Validate]: 100%|██████████| 747/747 [00:29<00:00, 25.29it/s]


[Validate] Atelectasis AUC: 0.7909 | F1: 0.3597
[Validate] Cardiomegaly AUC: 0.8510 | F1: 0.2997
[Validate] Consolidation AUC: 0.7934 | F1: 0.2161
[Validate] Edema AUC: 0.8481 | F1: 0.1732
[Validate] Effusion AUC: 0.8419 | F1: 0.4841
[Validate] Emphysema AUC: 0.8690 | F1: 0.3542
[Validate] Fibrosis AUC: 0.8118 | F1: 0.1358
[Validate] Hernia AUC: 0.8498 | F1: 0.1081
[Validate] Infiltration AUC: 0.6965 | F1: 0.4118
[Validate] Mass AUC: 0.8087 | F1: 0.3292
[Validate] Nodule AUC: 0.7274 | F1: 0.2507
[Validate] Pleural_Thickening AUC: 0.7641 | F1: 0.1933
[Validate] Pneumonia AUC: 0.6426 | F1: 0.0451
[Validate] Pneumothorax AUC: 0.8439 | F1: 0.3668
[Validate] Avg AUC: 0.7956, Avg F1: 0.2663


[Test]: 100%|██████████| 288/288 [00:11<00:00, 24.68it/s]


[Test] Atelectasis AUC: 0.7628 | F1: 0.3341
[Test] Cardiomegaly AUC: 0.8786 | F1: 0.3511
[Test] Consolidation AUC: 0.7518 | F1: 0.2047
[Test] Edema AUC: 0.8506 | F1: 0.2703
[Test] Effusion AUC: 0.8728 | F1: 0.5571
[Test] Emphysema AUC: 0.8983 | F1: 0.3537
[Test] Fibrosis AUC: 0.7623 | F1: 0.0960
[Test] Hernia AUC: 0.9081 | F1: 0.3571
[Test] Infiltration AUC: 0.6821 | F1: 0.3834
[Test] Mass AUC: 0.8454 | F1: 0.3789
[Test] Nodule AUC: 0.6763 | F1: 0.2286
[Test] Pleural_Thickening AUC: 0.7379 | F1: 0.2432
[Test] Pneumonia AUC: 0.7012 | F1: 0.0465
[Test] Pneumothorax AUC: 0.8497 | F1: 0.2788
[Test] Avg AUC: 0.7984, Avg F1: 0.2917


# Methods

## Data augmentation

##  Data preprocessing

## Neural Models

# Bayesian Baseline: GaussianNB

# EfficientNet - B4 + CBAM + 14 class threshold (maybe)

## Loss Function or a key product

### Cross entropy loss

### Focal loss -> focuses on hard to classify or rare examples

## Fitting

# Results

## Performance metric

### accuracy

### AUC

### F1 ->  multi-label classification and datasets with high class imbalance

## Comparison

## Model insight and robustness analysis (classification error, inputs that are problematic)

## Confidence

# Conclusion

## Generalizability

## Improvements

## take away for practitioners and for researchers